#### 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-customer-churn-prediction-spark4ai/sample_submission.csv
/kaggle/input/bank-customer-churn-prediction-spark4ai/train.csv
/kaggle/input/bank-customer-churn-prediction-spark4ai/test.csv


In [3]:
from IPython.display import display, Markdown
import numpy as np
def set_frame_style(df, caption="", font_size ='20px' ):
    random_list = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                      'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                      'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
    """Helper function to set dataframe presentation style.
    """
    return df.style.background_gradient(cmap=random_list[np.random.randint(1,17)]).set_caption(caption).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'Brown'),
        ('font-size', font_size),
        ('font-weight','bold')
    ]}])

In [4]:
train=pd.read_csv('/kaggle/input/bank-customer-churn-prediction-spark4ai/train.csv')
test=pd.read_csv('/kaggle/input/bank-customer-churn-prediction-spark4ai/test.csv')
submit=pd.read_csv('/kaggle/input/bank-customer-churn-prediction-spark4ai/sample_submission.csv')

**EDA**

In [5]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Features","Unique Values"]
for i in list(train.columns) :
    nunique =train[str(i)].nunique
    table.add_row([i, f"{nunique()}"])
print('Unique values in original dataset : \n')
print(table)

Unique values in original dataset : 

+-----------------+---------------+
|     Features    | Unique Values |
+-----------------+---------------+
|        id       |     15000     |
|    CustomerId   |      6293     |
|     Surname     |      764      |
|   CreditScore   |      379      |
|    Geography    |       3       |
|      Gender     |       2       |
|       Age       |       54      |
|      Tenure     |       12      |
|     Balance     |      3354     |
|  NumOfProducts  |       4       |
|    HasCrCard    |       2       |
|  IsActiveMember |       2       |
| EstimatedSalary |      6135     |
|      Exited     |       2       |
+-----------------+---------------+


In [6]:
set_frame_style(pd.DataFrame(train.Exited.value_counts()), 'Data points per class', '15px')

,count
Exited,
0.000000,11937
1.000000,3063


**FEATURE ENGINEERING**

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [8]:
train.drop(columns=['Surname','CustomerId'],inplace=True)
train.head()

,id,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,632.0,France,Female,63.0,5.0,0.00,2.0,0.0,1.0,140780.80,0.0
1,1,730.0,Germany,Female,35.0,5.0,182532.23,2.0,1.0,1.0,136491.32,0.0
2,2,598.0,Spain,Male,45.0,2.0,94049.92,1.0,1.0,1.0,131718.12,0.0
3,3,678.0,France,Female,38.0,9.0,0.00,2.0,1.0,1.0,164768.82,0.0
4,4,710.0,France,Female,32.0,4.0,0.00,2.0,0.0,0.0,100187.43,0.0


In [9]:
train['BalanceSalaryRatio'] = train.Balance/(train.EstimatedSalary)
train['TenureByAge'] = train.Tenure/(train.Age)
train['CreditScoreGivenAge'] = train.CreditScore/(train.Age)
train['Age_Category'] = pd.cut(train['Age'], bins=[18, 30, 40, 50, 60, 100], labels=['18-30', '30-40', '40-50', '50-60', '60+'])
    

In [10]:
test['BalanceSalaryRatio'] = test.Balance/test.EstimatedSalary
test['TenureByAge'] = test.Tenure/(test.Age)
test['CreditScoreGivenAge'] = test.CreditScore/(test.Age)
test['Age_Category'] = pd.cut(test['Age'], bins=[18, 30, 40, 50, 60, 100], labels=['18-30', '30-40', '40-50', '50-60', '60+'])
   

**REARRANGING AND SPLITTING THE DATA**

In [11]:
train.dropna(axis=0,subset=['Exited'],inplace=True)
y=train.Exited
train.drop(columns=['Exited'],axis=1,inplace=True)

In [12]:
X_train,X_valid,Y_train,Y_valid=train_test_split(train,y,train_size=0.8,random_state=0)

In [13]:
X_train.head()

,id,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,BalanceSalaryRatio,TenureByAge,CreditScoreGivenAge,Age_Category
13136,13136,637.0,Germany,Female,42.0,9.0,123082.08,1.0,1.0,0.0,53949.98,2.281411,0.214286,15.166667,40-50
3536,3536,623.0,France,Male,33.0,6.0,0.00,1.0,1.0,0.0,121061.89,0.000000,0.181818,18.878788,30-40
10296,10296,667.0,France,Female,30.0,8.0,0.00,2.0,0.0,1.0,16081.62,0.000000,0.266667,22.233333,18-30
2137,2137,628.0,France,Male,26.0,8.0,0.00,2.0,1.0,1.0,156424.40,0.000000,0.307692,24.153846,18-30
14775,14775,593.0,France,Male,34.0,4.0,152324.66,1.0,1.0,0.0,110440.50,1.379246,0.117647,17.441176,30-40


In [14]:
X_valid.head()

,id,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,BalanceSalaryRatio,TenureByAge,CreditScoreGivenAge,Age_Category
1670,1670,591.0,Spain,Male,36.0,10.0,132271.50,1.0,1.0,1.0,175071.29,0.755529,0.277778,16.416667,30-40
13379,13379,778.0,France,Male,25.0,9.0,0.00,2.0,1.0,1.0,145558.42,0.000000,0.360000,31.120000,18-30
10234,10234,730.0,Spain,Female,44.0,7.0,120124.35,2.0,1.0,1.0,166678.28,0.720696,0.159091,16.590909,40-50
4719,4719,681.0,France,Male,26.0,7.0,0.00,1.0,1.0,0.0,122379.54,0.000000,0.269231,26.192308,18-30
7003,7003,667.0,Germany,Female,22.0,2.0,136416.84,2.0,1.0,1.0,189143.94,0.721233,0.090909,30.318182,18-30


In [15]:
categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == "object"]


numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]
continuous_vars=numerical_cols[1:]
print(continuous_vars)


['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'BalanceSalaryRatio', 'TenureByAge', 'CreditScoreGivenAge']


In [16]:
my_cols = categorical_cols + numerical_cols
X_train1 = X_train[my_cols].copy()
X_valid1 = X_valid[my_cols].copy()
X_test1 = test[my_cols].copy()

**DATA PREPROCESSING AND TRAINING USING GRADIENT BOOSTING CLASSIFIER**

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from xgboost import XGBClassifier

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
# categorical_transformer = Pipeline(steps=[
#     ('labelencoder', LabelEncoder()),  # Label encoding for categorical features
#     ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
# ])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model =  GradientBoostingClassifier()
# model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,colsample_bytree=1, gamma=0.01, learning_rate=0.1, max_delta_step=0,max_depth=7,
#                     min_child_weight=5, missing=None, n_estimators=20,n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,reg_alpha=0, 
#                     reg_lambda=1, scale_pos_weight=1, seed=None, silent=True, subsample=1)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ("scale",QuantileTransformer(n_quantiles=200)),
                      ('classifier', model),
                     ])

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(Y_train)
# Preprocessing of training data, fit model 
clf.fit(X_train1,Y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid1)

print('MAE:', mean_absolute_error(Y_valid, preds))

MAE: 0.09466666666666666


**PERFORMING CROSS VALIDATION**

In [18]:
# from sklearn.model_selection import GridSearchCV
# gb_classifier = GradientBoostingClassifier()

# # Define hyperparameters to tune
# param_grid = {
#     'classifier__n_estimators': [50, 100, 200],
#     'classifier__learning_rate': [0.01, 0.1, 0.5],
#     'classifier__max_depth': [3, 5, 7],
#     'classifier__min_samples_split': [2, 5, 10],
#     'classifier__min_samples_leaf': [1, 2, 4],
# }

# # Grid search with cross-validation
# grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# # Fit the grid search to the data
# grid_search.fit(X_train, Y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Score:", best_score)

# best_model = grid_search.best_estimator_
# test_score = best_model.score(X_test, Y_test)
# print("Test Set Score:", test_score)

**PREDICTING PROBABILITIES**

In [19]:
pred1=pd.DataFrame(clf.predict_proba(X_test1))[1]

In [20]:
pred1

0       0.010920
1       0.285588
2       0.032994
3       0.954521
4       0.016552
          ...   
9995    0.954961
9996    0.055300
9997    0.419409
9998    0.740955
9999    0.205552
Name: 1, Length: 10000, dtype: float64

In [21]:
submit

,id,Exited
0,15000,0.5
1,15001,0.5
2,15002,0.5
3,15003,0.5
4,15004,0.5
...,...,...
9995,24995,0.5
9996,24996,0.5
9997,24997,0.5
9998,24998,0.5


In [22]:
test['Exited']=pred1
new=test.loc[:,['id','Exited']]

In [23]:
new

,id,Exited
0,15000,0.010920
1,15001,0.285588
2,15002,0.032994
3,15003,0.954521
4,15004,0.016552
...,...,...
9995,24995,0.954961
9996,24996,0.055300
9997,24997,0.419409
9998,24998,0.740955


In [24]:

new.to_csv('submission.csv', index=False)

In [25]:
# from xgboost import XGBRegressor

# X_train = pd.get_dummies(X_train)
# X_valid = pd.get_dummies(X_valid)
# X_test1= pd.get_dummies(X_test1)
# X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
# X_train, X_test1 = X_train.align(X_test1, join='left', axis=1)


In [ ]:
# Define model
# model = XGBRegressor(n_estimators=5000, learning_rate=0.001, n_jobs=4)




# model.fit(X_train, Y_train, 
#              early_stopping_rounds=2, 
#              eval_set=[(X_valid, Y_valid)], 
#              verbose=False)
